In [ ]:
import pandas as pd
from tqdm.auto import tqdm

pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 500

tqdm.pandas()

## load data

In [ ]:
df = pd.read_csv('maupqa_v2.1.csv.gz')
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df['score_unicamp-dl/mt5-3B-mmarco-en-pt'] = df['score_unicamp-dl/mt5-3B-mmarco-en-pt'].apply(lambda x: min(x, 1.0))
df['score_unicamp-dl/mt5-3B-mmarco-en-pt'] = df['score_unicamp-dl/mt5-3B-mmarco-en-pt'].apply(lambda x: max(x, 0.0))

In [ ]:
df['subset'].value_counts()

## question blacklist

In [ ]:
wrong_questions = {
    'Co to znaczy?',
    'Co się dzieje?',
    'Co się stało?',
    'O co chodzi?',
    'Co masz na myśli?',

    'Ile odcinków będzie w 1. sezonie?',
    'Kiedy zaczyna się nowy sezon?',
    'Kiedy wychodzi nowy film?',
    'Kiedy zaczyna się sezon 3?',
    'Kto zagra w nowym filmie?',
    'Ile odcinków jest w 1. sezonie?',
    'Ile odcinków będzie w 2. sezonie?',
    'Kiedy zaczyna się sezon 5?',
    'Gdzie film został nakręcony?',
    'Gdzie kręcono ten film?',
    'Kiedy to wyszło?',
    'Gdzie kręcono serial?',
    'Czy marlon Brando pojawił się w filmie?',
    
    'Kto napisał tę piosenkę?',
    'Kto śpiewa tę piosenkę?',
    'Kto zaśpiewał tę piosenkę?',
    'Kto śpiewał tę piosenkę?',
    'Kto śpiewa piosenkę?',
    'Kto napisał piosenkę?',
    'Kto zaśpiewał oryginał?',
    'Kto tak śpiewa?',
    'Kto zaśpiewał piosenkę?',
    'Kto stworzył tę piosenkę?',
    'Kiedy piosenka została wydana?',
    'Kto grał na gitarze?',
    'Kiedy pojawiła się ta piosenka?',
    'Kto śpiewa piosenkę country?',
    'Kto to napisał?',
    
    'Co przedstawia ten paragraf?',
    'Jaki jest temat paragrafu?',
    'Co zawiera paragraf?',
    'Jakie książki zostały wymienione w paragrafie?',
    'Jakie informacje zawiera paragraf?',
    'Co opisuje ten paragraf?',
    
    'Zła definicja.',
    'Słaba definicja.',
    'Zgodnie z definicją.',
    'Błędna definicja.',
    
    'Co to za hrabstwo?',
    'W jakim hrabstwie.',
    'W jakim hrabstwie?',
    
    'Gdzie leży Kalifornia?',
    'Gdzie jest Kalifornia?',
    
    'Gdzie mogę znaleźć pełną instrukcję obsługi do swojego samochodu?',
    'Gdzie zaczyna się i kończy rzeka?',
    'Co to za imię?',
    'Kto chce zostać milionerem?',
    'Jaki poziom ewoluuje?',
    'To ma znaczenie.',
    'Jakie dokumenty są potrzebne?',
    'Kto to jest?',
    'Skąd się wzięło to wyrażenie?',
    'Jakie to uczucie?',
    'Co oznacza ustawa?',
}

df = df[~df['question'].isin(wrong_questions)]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## words blacklist

In [ ]:
blacklist = {
    'definicja', 'zdefiniuj', 'zdefiniować',
    'paragraf', 'paragrafu', 'paragrafie',
    'pogoda',
}

def count_blacklist(text):
    text = text.lower().replace('.', '').replace(',', '').replace('?', '')
    return len(set(text.split(' ')) & blacklist)


df['blacklist'] = df['question'].progress_apply(count_blacklist) > 0

In [ ]:
df = df[~df['blacklist']]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## short question

In [ ]:
df['q_clen'] = df['question'].apply(len)
df['q_wlen'] = df['question'].apply(lambda x: x.count(' ') + 1)

In [ ]:
df = df[~(df['relevant'] & (df['q_clen'] < 10))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~(df['relevant'] & (df['q_wlen'] < 3))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## long question

In [ ]:
df = df[~(df['relevant'] & (df['q_clen'] > 300))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~(df['relevant'] & (df['q_wlen'] > 50))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## short passage

In [ ]:
df['p_clen'] = df['passage_text'].apply(len)
df['p_wlen'] = df['passage_text'].apply(lambda x: x.count(' ') + 1)

In [ ]:
df = df[~(df['relevant'] & (df['p_clen'] < 30))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~(df['relevant'] & (df['p_wlen'] < 5))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## long passage

In [ ]:
df = df[~(df['relevant'] & (df['p_clen'] > 1000))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~(df['relevant'] & (df['p_wlen'] > 150))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## ce score

In [ ]:
df = df[~((df['score_unicamp-dl/mt5-3B-mmarco-en-pt'] < 0.1) & (df['relevant'] == True))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~((df['score_unicamp-dl/mt5-3B-mmarco-en-pt'] > 0.9) & (df['relevant'] == False))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## encoder score

In [ ]:
df = df[~((df['score_intfloat/multilingual-e5-base'] < 0.75) & (df['relevant'] == True))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
df = df[~((df['score_intfloat/multilingual-e5-base'] > 0.9) & (df['relevant'] == False))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

In [ ]:
# too easy
df = df[~(df['relevant'] & (df['score_intfloat/multilingual-e5-base'] > 0.95))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## redirect

In [ ]:
df = df[~(df['relevant'] & df['passage_text'].str.startswith('redirect'))]
df = df[~(df['relevant'] & df['passage_text'].str.startswith('Redrect'))]
df = df[~(df['relevant'] & df['passage_text'].str.startswith('REDIRECT'))]
df = df[~(df['relevant'] & df['passage_text'].str.startswith('PATRZ'))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## common passages

In [ ]:
passage_counts = df.loc[df['relevant'], 'passage_text'].value_counts().reset_index()
common_passages = passage_counts.loc[passage_counts['count'] > 10, 'passage_text'].values

df = df[~(df['relevant'] & (df['passage_text'].isin(common_passages)))]
df[df['relevant']].shape, df.loc[df['relevant'], 'question'].unique().shape

## save

In [ ]:
df[[
    'question_id', 'question', 'answers', 
    'passage_title', 'passage_text', 'relevant', 
    'passage_source', 'subset',
]].to_csv('maupqa_v2.1_clean.csv.gz', index=False)